In [ ]:
import json
import pandas as pd
from bertopic import BERTopic

# --- 1. CARREGAR DADOS (Parte Rápida) ---
ARQUIVO_ALVO = "dataset_Marvel.json"
print(f"Carregando dados de: {ARQUIVO_ALVO}...")

with open(ARQUIVO_ALVO, "r", encoding="utf-8") as f:
    data = json.load(f)

todos_comentarios = []
for movie_name, movie_data in data.items():
    comments_list = movie_data.get("comments", [])
    for comment in comments_list:
        texto = comment.get("text", "")
        if len(texto) > 10: 
            todos_comentarios.append(texto)

print(f"Total carregado: {len(todos_comentarios)} comentários.")

# --- 2. TREINAR E SALVAR (Parte Demorada - A Noite Toda) ---
print("Iniciando o treino... Pode ir dormir.")

# Configura
topic_model = BERTopic(language="multilingual", verbose=True)

# Treina (Vai demorar horas)
topics, probs = topic_model.fit_transform(todos_comentarios)

# --- 3. SALVAMENTO DE SEGURANÇA (O Pulo do Gato) ---
print("Treino finalizado! Salvando o cérebro do modelo...")
topic_model.save("modelo_marvel_final")

# Salva também a tabela em Excel/CSV para garantir
info_topicos = topic_model.get_topic_info()
nome_saida = "tabela_topicos_marvel.csv"
info_topicos.to_csv(nome_saida, index=False)

print(f"TUDO PRONTO! Modelo salvo em 'modelo_marvel_final' e tabela em '{nome_saida}'")

In [ ]:
import json
import pandas as pd
from bertopic import BERTopic

# --- 1. CARREGAR DADOS DA DC ---
ARQUIVO_ALVO = "dataset_DC.json"
print(f"Carregando dados de: {ARQUIVO_ALVO}...")

with open(ARQUIVO_ALVO, "r", encoding="utf-8") as f:
    data = json.load(f)

todos_comentarios_dc = []
for movie_name, movie_data in data.items():
    comments_list = movie_data.get("comments", [])
    for comment in comments_list:
        texto = comment.get("text", "")
        # Filtro básico: ignora comentários muito curtos
        if len(texto) > 10: 
            todos_comentarios_dc.append(texto)

print(f"Total carregado (DC): {len(todos_comentarios_dc)} comentários.")

# --- 2. TREINAR E SALVAR (Pode demorar algumas horas) ---
print("Iniciando o treino para DC...")

# Configura
topic_model_dc = BERTopic(language="multilingual", verbose=True)

# Treina
topics_dc, probs_dc = topic_model_dc.fit_transform(todos_comentarios_dc)

# --- 3. SALVAMENTO ---
print("Treino DC finalizado! Salvando modelo...")
topic_model_dc.save("modelo_dc_final")

# Salva a tabela
info_topicos_dc = topic_model_dc.get_topic_info()
nome_saida = "tabela_topicos_dc.csv"
info_topicos_dc.to_csv(nome_saida, index=False)

print(f"TUDO PRONTO! Modelo salvo em 'modelo_dc_final' e tabela em '{nome_saida}'")

In [1]:
import json
from bertopic import BERTopic

# 1. Recarregar os dados do arquivo (Mude para dataset_DC.json se for analisar a DC)
with open("dataset_Marvel.json", "r", encoding="utf-8") as f:
    data = json.load(f)

todos_comentarios = [c.get("text", "") for m in data.values() for c in m.get("comments", []) if len(c.get("text", "")) > 10]

# 2. Carregar o modelo e reduzir
topic_model = BERTopic.load("modelo_marvel_final")
topic_model.reduce_topics(todos_comentarios, nr_topics=15)

# 3. Gerar o gráfico
topic_model.visualize_barchart(top_n_topics=15)

In [3]:
import json
from bertopic import BERTopic

# 1. Recarregar os textos da DC (Garante que 'todos_comentarios_dc' existe)
print("Carregando textos da DC...")
with open("dataset_DC.json", "r", encoding="utf-8") as f:
    data_dc = json.load(f)

todos_comentarios_dc = []
for movie in data_dc.values():
    for comment in movie.get("comments", []):
        texto = comment.get("text", "")
        if len(texto) > 10:
            todos_comentarios_dc.append(texto)

# 2. Carregar o modelo salvo
print("Carregando modelo...")
topic_model_dc = BERTopic.load("modelo_dc_final")

# 3. Reduzir para 10 tópicos para facilitar a análise
print("Simplificando tópicos...")
topic_model_dc.reduce_topics(todos_comentarios_dc, nr_topics=10)

# 4. Gerar e salvar o gráfico
print("Gerando gráfico...")
fig = topic_model_dc.visualize_barchart(top_n_topics=10, title="Principais Temas: DC")
fig.show()

# Opcional: Salvar o gráfico como imagem para o seu relatório
# fig.write_image("grafico_dc_temas.png")

Carregando textos da DC...
Carregando modelo...
Simplificando tópicos...
Gerando gráfico...


In [2]:
import json
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic import BERTopic

# 1. PREPARAÇÃO DE LIMPEZA (Stopwords)
nltk.download('stopwords')
from nltk.corpus import stopwords

# Criando a lista mestre de palavras inúteis
stops = list(set(stopwords.words('portuguese') + stopwords.words('english') + stopwords.words('spanish')))
lixo_geral = ["video", "movie", "film", "dc", "marvel", "trailer", "trailers", "filme", "filmes", "quem", "acho", "can", "get", "see"]
# Adicionando os novos lixos que você identificou
lixo_especifico = [
    "warranty", "extend", "cars", "thanks", "thank", "explained", "explanation", 
    "sharing", "rabbi", "orthodox", "hanuman", "lord", "ji", "avm23x", 
    "retention", "engagement", "google", "googling", "looks", "wait", "good", "best"
]
final_stops = stops + lixo_geral + lixo_especifico

# 2. CARREGAMENTO DOS DADOS E MODELO
print("Carregando arquivos...")
with open("dataset_DC.json", "r", encoding="utf-8") as f:
    data_dc = json.load(f)

# Reconstruindo a lista de comentários para o reduce_topics
todos_comentarios_dc = [
    c.get("text", "") for m in data_dc.values() 
    for c in m.get("comments", []) if len(c.get("text", "")) > 10
]

topic_model_dc = BERTopic.load("modelo_dc_final")

# 3. APLICAÇÃO DA LIMPEZA E REDUÇÃO
print("Limpando e reduzindo tópicos...")
vectorizer_model = CountVectorizer(stop_words=final_stops)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

# Atualiza os nomes dos tópicos sem re-treinar o modelo
topic_model_dc.update_topics(
    todos_comentarios_dc, 
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model
)

# Agrupa em 12 grandes temas para facilitar seu relatório
topic_model_dc.reduce_topics(todos_comentarios_dc, nr_topics=12)

# 4. GERAÇÃO E SALVAMENTO DO GRÁFICO
print("Gerando visualizações...")
fig = topic_model_dc.visualize_barchart(
    top_n_topics=12, 
    n_words=8, 
    title="Análise Temática: Universo DC (2000-2025)"
)

# Salva como HTML (Interativo para você ver no navegador)
fig.write_html("grafico_dc_interativo.html")

# Tenta salvar como imagem estática (Para o Word/Relatório)
# Nota: Pode precisar de 'pip install -U kaleido' se der erro aqui
try:
    fig.write_image("grafico_dc_final.png")
    print("Sucesso! Gráfico salvo como 'grafico_dc_final.png'")
except:
    print("Aviso: Não foi possível salvar como PNG. Use o arquivo HTML e tire um print.")

fig.show()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Carregando arquivos...
Limpando e reduzindo tópicos...
Gerando visualizações...
Aviso: Não foi possível salvar como PNG. Use o arquivo HTML e tire um print.


In [4]:
import json
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

# 1. Lista de limpeza atualizada com os novos lixos que você viu
lixo_spam = [
    "content", "tips", "informative", "helpful", "needed", "16", "land",
    "thanks", "thank", "explained", "explanation", "sharing", "warranty", 
    "extend", "reach", "cars", "googling", "algorithm", "avm23x"
]

# Palavras que não trazem tema (Stopwords)
from nltk.corpus import stopwords
nltk.download('stopwords')
final_stops = list(set(stopwords.words('portuguese') + stopwords.words('english'))) + lixo_spam

# 2. Carregar dados e modelo
with open("dataset_DC.json", "r", encoding="utf-8") as f:
    data_dc = json.load(f)
textos_dc = [c.get("text", "") for m in data_dc.values() for c in m.get("comments", []) if len(c.get("text", "")) > 10]

topic_model_dc = BERTopic.load("modelo_dc_final")

# 3. Atualizar e Reduzir
vectorizer_model = CountVectorizer(stop_words=final_stops)
topic_model_dc.update_topics(textos_dc, vectorizer_model=vectorizer_model)

# Vamos manter 10 tópicos para não misturar tanto "Superman" com "Aquaman"
topic_model_dc.reduce_topics(textos_dc, nr_topics=10)

# 4. Salvar e Mostrar
fig = topic_model_dc.visualize_barchart(top_n_topics=10, title="DC: Análise de Temas Real")
fig.show()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import json
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

################################################################################################################


                                #   NARRATIVA: A DC FOCA NOS FILMES EM SI   #                                   


################################################################################################################

# --- CONFIGURAÇÃO INICIAL (Igual ao anterior) ---
nltk.download('stopwords')
from nltk.corpus import stopwords

# Lista de limpeza focada na narrativa MICRO
stops_padrao = list(set(stopwords.words('portuguese') + stopwords.words('english') + stopwords.words('spanish')))
palavras_vazias = [
    "movie", "film", "trailer", "dc", "dceu", "wb", "warner", "bros", "scene", 
    "good", "bad", "best", "love", "like", "watch", "watching", "wait", "hype",
    "looks", "looking", "think", "make", "see", "know", "people", "guy", "one",
    "warranty", "cars", "part", "time", "video", "content"
]
final_stops = stops_padrao + palavras_vazias

print("Carregando dados e modelo da DC...")
with open("dataset_DC.json", "r", encoding="utf-8") as f:
    data_dc = json.load(f)
textos_dc = [c.get("text", "") for m in data_dc.values() for c in m.get("comments", []) if len(c.get("text", "")) > 10]

topic_model_dc = BERTopic.load("modelo_dc_final")

print("Aplicando filtro de foco temático...")
vectorizer_model = CountVectorizer(stop_words=final_stops)
topic_model_dc.update_topics(textos_dc, vectorizer_model=vectorizer_model)

# Tópicos selecionados para a narrativa "MICRO/FILME"
topicos_alvo = {
    1: "Lady Gaga e Joker 2 (Foco em Elenco)",
    2: "Batman Robert Pattinson (Estética/Brutal)",
    3: "Aquaman (Discussão do Filme)",
    4: "Joker (O Personagem)",
    9: "Animação (Estética Visual)",
    16: "Superman (Legado do Personagem)",
    17: "Shazam (Tom do Filme)"
}

# --- GERAÇÃO COM COR AZUL ---
print("Gerando gráficos estilizados (Azul DC)...")
# Cor Azul DC (Um azul escuro e sóbrio)
COR_DC = "#0B08B4" 

for topic_id, nome_descritivo in topicos_alvo.items():
    try:
        # 1. Gera o gráfico base
        fig = topic_model_dc.visualize_barchart(
            topics=[topic_id], 
            n_words=10, 
            title=f"DC Tópico {topic_id}: {nome_descritivo}"
        )
        
        # 2. APLICA A COR AZUL NAS BARRAS
        # Isso atualiza a cor dos "marcadores" (as barras) do gráfico
        fig.update_traces(marker_color=COR_DC)
        
        # (Opcional) Deixa o título azul também para combinar
        fig.update_layout(title_font_color=COR_DC)
        
        # 3. Salva o arquivo
        filename = f"DC_Azul_Topico_{topic_id}_{nome_descritivo.split('(')[0].strip().replace(' ', '_')}.png"
        fig.write_image(filename)
        print(f"Salvo: {filename}")
        
    except Exception as e:
        print(f"Erro ao gerar tópico {topic_id}: {e}")

print("Concluído! Verifique as imagens azuis na pasta.")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Carregando dados e modelo da DC...
Aplicando filtro de foco temático...
Gerando gráficos estilizados (Azul DC)...
Salvo: DC_Azul_Topico_1_Lady_Gaga_e_Joker_2.png
Salvo: DC_Azul_Topico_2_Batman_Robert_Pattinson.png
Salvo: DC_Azul_Topico_3_Aquaman.png
Salvo: DC_Azul_Topico_4_Joker.png
Salvo: DC_Azul_Topico_9_Animação.png
Salvo: DC_Azul_Topico_16_Superman.png
Salvo: DC_Azul_Topico_17_Shazam.png
Concluído! Verifique as imagens azuis na pasta.
